In [1]:
import random

def initialize(N=8): #random initialization
    return [random.randint(1, N) for _ in range(N)]

#test
queens = initialize()
print(queens)

[8, 3, 6, 7, 3, 8, 3, 5]


In [3]:
def heuristic(state, N=8): # attacking queens heuristic function
    cost = 0
    for i in range(N):
        for j in range(i+1, N):
            if state[i] == state[j] or abs(j-i) == abs(state[j]-state[i]):
                cost +=1
    return cost

#test
cost = heuristic(queens)
print(cost)

6


In [4]:
def printTable(state, N=8):
    table = [['-' for _ in range(N)] for _ in range(N)]
    for column, row in enumerate(state):
        table[row-1][column] = 'Q'
    for i in range(N):
        for j in range(N):
            print(table[i][j], end='  ')
        print()

#test
printTable(queens)

-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  
-  Q  -  -  Q  -  Q  -  
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  Q  
-  -  Q  -  -  -  -  -  
-  -  -  Q  -  -  -  -  
Q  -  -  -  -  Q  -  -  


In [5]:
def successors(state, N=8): # returns all successors
    Successors = []
    for i in range(N):
        for j in range(1, N+1):
            newState = state.copy()
            newState[i] = j if newState[i] != j else 0
            if 1 <= newState[i] <= N:
                Successors.append(newState)
    return Successors

#test
suc = successors(queens)
print(len(suc))

56


# Hill Climbing

In [9]:
def steepestAscent(initState):

    if heuristic(initState) == 0:
        return initState, True

    def minsuccessor(state):
        states = successors(state)
        statesWithCost = [(state, heuristic(state)) for state in states]
        minval = min(statesWithCost, key = lambda x : x[1])[1]
        minlist = [x[0] for x in statesWithCost if x[1] == minval]
        return random.choice(minlist)
    
    prevState = initState
    while True:
        state = minsuccessor(prevState)
        if heuristic(state) >= heuristic(prevState):
            return prevState, False
        elif heuristic(state) == 0:
            return state, True
        prevState = state

#test
solution = steepestAscent(queens)
print(solution[1])
printTable(solution[0])

False
-  Q  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  -  Q  -  Q  -  
-  -  Q  -  -  -  -  -  
-  -  -  -  -  -  -  Q  
-  -  -  -  -  Q  -  -  
-  -  -  Q  -  -  -  -  
Q  -  -  -  -  -  -  -  


In [6]:
def firstChoice(initState):
  
    if heuristic(initState) == 0:
        return initState, True

    def firstsuccessor(state):
        states = successors(state)
        for childstate in states:
            if heuristic(childstate) <= heuristic(state):
                return childstate
        return states[0]
    
    prevState = initState
    while True:
        state = firstsuccessor(prevState)
        if heuristic(state) >= heuristic(prevState):
            return prevState, False
        elif heuristic(state) == 0:
            return state, True
        prevState = state

#test
solution = firstChoice(queens)
print(solution[1])
printTable(solution[0])

False
-  -  Q  -  -  -  -  -  
-  -  -  -  -  -  Q  -  
-  -  -  -  -  -  -  -  
-  -  -  -  -  Q  -  -  
Q  -  -  Q  -  -  -  -  
-  -  -  -  Q  -  -  -  
-  -  -  -  -  -  -  -  
-  Q  -  -  -  -  -  Q  


In [7]:
def randomRestart(initState):
    while True:
        solution = steepestAscent(initState)
        if solution[1]:
            return solution[0]
        initState = initialize()

#test
solution = randomRestart(queens)
printTable(solution)

-  -  -  -  Q  -  -  -  
-  -  Q  -  -  -  -  -  
Q  -  -  -  -  -  -  -  
-  -  -  -  -  -  Q  -  
-  Q  -  -  -  -  -  -  
-  -  -  -  -  -  -  Q  
-  -  -  -  -  Q  -  -  
-  -  -  Q  -  -  -  -  


# Simulated Anealing

In [8]:
from math import exp
def simAnnealing(initState):

    def randomSuccessor(state):
        states = successors(state)
        return random.choice(states)
    
    current = initState
    Temperatue = 100
    cool = 0.98
    while True:
        Temperatue *= cool
        if heuristic(current) == 0:
            return current
        nextState = randomSuccessor(current)
        deltaE = heuristic(nextState) - heuristic(current)
        if deltaE < 0:
            current = nextState
        else:
            threshhold = exp(-deltaE/Temperatue)
            prob = random.random()
            if prob < threshhold:
                current = nextState
            if heuristic(current) == 0:
              return current

#test
solution = simAnnealing(queens)
printTable(solution)

-  -  -  -  Q  -  -  -  
-  -  -  -  -  -  Q  -  
Q  -  -  -  -  -  -  -  
-  -  Q  -  -  -  -  -  
-  -  -  -  -  -  -  Q  
-  -  -  -  -  Q  -  -  
-  -  -  Q  -  -  -  -  
-  Q  -  -  -  -  -  -  


In [9]:
def main():
    initState = initialize()
    print('Initial State:')
    printTable(initState)
    print(f'cost: {heuristic(initState)}')

    solution = steepestAscent(initState)
    if solution[1]:
        print('Steepest Ascent solution: ')
        printTable(solution[0])
        print(f'cost: {heuristic(solution[0])}')
    else:
        print('Steepest Ascent stuck in local minima: ')
        printTable(solution[0])
        print(f'cost: {heuristic(solution[0])}')

    solution = firstChoice(initState)
    if solution[1]:
        print('First Choice solution: ')
        printTable(solution[0])
        print(f'cost: {heuristic(solution[0])}')
    else:
        print('First Choice stuck in local minima: ')
        printTable(solution[0])
        print(f'cost: {heuristic(solution[0])}')

    solution = randomRestart(initState)
    print('Random Restart solution: ')
    printTable(solution)
    print(f'cost: {heuristic(solution)}')

    solution = simAnnealing(initState)
    print('simulated Annealing solution: ')
    printTable(solution)
    print(f'cost: {heuristic(solution)}')

if __name__ == '__main__':
    main()

Initial State:
-  -  -  -  Q  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  
Q  -  Q  -  -  Q  Q  -  
-  Q  -  Q  -  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  Q  
cost: 10
Steepest Ascent stuck in local minima: 
-  -  -  -  Q  -  -  -  
-  -  Q  -  -  -  -  -  
Q  -  -  -  -  -  -  -  
-  -  -  -  -  Q  Q  -  
-  -  -  Q  -  -  -  -  
-  Q  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  Q  
cost: 1
First Choice stuck in local minima: 
Q  -  -  -  Q  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  
-  -  Q  -  -  Q  Q  -  
-  Q  -  Q  -  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  -  
-  -  -  -  -  -  -  Q  
cost: 8
Random Restart solution: 
-  -  -  Q  -  -  -  -  
-  Q  -  -  -  -  -  -  
-  -  -  -  -  -  -  Q  
-  -  -  -  -  Q  -  -  
Q  -  -  -  -  -  -  -  
-  -  Q  -  -  -  -  -  
-  -  -  -  Q  -  -  -  
-  -  -  -  -  -  Q  -  
cost: 0
simulated Annealing solution: 
-  -  -  Q  -  -  